In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
from scipy.stats import norm
import scipy.integrate as integrate

import gym
from gym import spaces

import random
import itertools as it
from joblib import Parallel, delayed
from toolz import memoize
from contracts import contract
from collections import namedtuple, defaultdict, deque, Counter

import warnings
warnings.filterwarnings("ignore", 
                        message="The objective has been evaluated at this point before.")

from agents import Agent
from oldmouselab import OldMouselabEnv
from policies import FixedPlanPolicy, LiederPolicy
from evaluation import *
from distributions import cmax, smax, sample, expectation, Normal, PointMass, SampleDist, Normal, Categorical

In [2]:
def hd_dist(attributes):
    dist = [1,]*attributes
    dist[0] = np.random.randint(85,97)
    for i in range(1,attributes-1):
        dist[i] += np.random.randint(0,100-np.sum(dist))
    dist[-1] += 100-np.sum(dist)
    dist = np.around(np.array(dist)/100,decimals=2)
    np.random.shuffle(dist)
    return dist

def ld_dist(attributes):
    dist = [np.random.randint(10,40) for _ in range(attributes)]
    dist = np.around(np.array(dist)/sum(dist),decimals=2)
    np.random.shuffle(dist)
    return dist

In [3]:
gambles = 7
attributes = 4
high_stakes = Normal((9.99+0.01)/2, 0.3*(9.99-0.01))
low_stakes = Normal((0.25+0.01)/2, 0.3*(0.25-0.01))
reward = high_stakes
cost=.03

#set to 20 for sanity check
n_train = 20
n_test = 20

train_envs_hd = [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
train_envs = train_envs_hd+train_envs_ld 

test_envs_hd =  [OldMouselabEnv(gambles, hd_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs_ld = [OldMouselabEnv(gambles, ld_dist(attributes), reward, cost) for _ in range(n_train)]
test_envs = test_envs_hd+test_envs_ld 

term_action = train_envs[0].term_action

In [4]:
bo_pol_theta = np.load('data/high_stakes_3cents.npy')
bo_pol = LiederPolicy(list(bo_pol_theta))

In [5]:
agent = Agent()
def run_env(policy, env):
    agent.register(env)
    agent.register(policy)
    tr = agent.run_episode()
#     print(tr)
    return {'util': tr['return'], 'actions': tr['actions'],
            'observations': len(tr['actions']) - 1, 'ground_truth': env.ground_truth}

In [6]:
train_envs[0].reset()
run_env(bo_pol, train_envs[0]) 

{'actions': [25, 5, 17, 1, 13, 21, 9, 28],
 'ground_truth': array([  2.488,   1.276,   3.603,   9.768,   7.232,   2.05 ,   6.453,   3.386,   7.488,   8.336,   7.65 ,   6.292,  -1.571,   0.272,   2.038,  11.036,  11.064,   3.955,  10.761,   1.996,   7.284,
          5.289,   5.209,   8.437,  -2.915,   8.968,   2.648,   6.39 ]),
 'observations': 7,
 'util': 8.4402693386115661}

In [7]:
train_envs[0].grid()

array([[Norm(5.00, 2.99), 1.27619283437495, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), 2.0497437710003212, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), 8.3357042409567512, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), 0.27201144805139688, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), 3.9546606020444122, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), 5.2886758654802524, Norm(5.00, 2.99), Norm(5.00, 2.99)],
       [Norm(5.00, 2.99), 8.9676840637082229, Norm(5.00, 2.99), Norm(5.00, 2.99)]], dtype=object)